In [1]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Neo4jVector
from langchain_community.chat_models import ChatOpenAI

# from langchain_community.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_community.graphs import Neo4jGraph
from langchain_community.chains.graph_qa.cypher import GraphCypherQAChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain_experimental.graph_transformers import LLMGraphTransformer
import streamlit as st
import tempfile
from neo4j import GraphDatabase

In [2]:
load_dotenv()

True

In [3]:
api_key = os.getenv("OPENAI_API_KEY")
llm_model_name = os.getenv("LLM_MODEL")
llm_model = ChatOpenAI(model_name=llm_model_name)

embedding_model_name = os.getenv("EMBEDDING_MODEL")

if embedding_model_name == "OpenAI":
    embedding_model = OpenAIEmbeddings()
else:
    raise ValueError("Unsupported embedding model specified in .env")

/var/folders/kt/9_rb87_95nl_6glc6tz14j9m0000gn/T/ipykernel_91978/2112057868.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm_model = ChatOpenAI(model_name=llm_model_name)


In [4]:
neo4j_uri = os.getenv("NEO4J_URI")
neo4j_username = os.getenv("NEO4J_USERNAME")
neo4j_password = os.getenv("NEO4J_PASSWORD")
neo4j_database = os.getenv("NEO4J_DATABASE")

graph = Neo4jGraph(
    url=neo4j_uri,
    username=neo4j_username,
    password=neo4j_password,
    database=neo4j_database,
)

In [30]:
# Load and split the PDF
# pdf_file_path = os.getenv("PDF_FILE_PATH")

# 1. 붙임1. 2024학년도 1학기 수강신청 안내.pdf
# pdf_file_path = "./input/붙임1. 2024학년도 1학기 수강신청 안내.pdf"

# 2. 붙임1. 2024학년도 1학기 신편입생 수강신청 안내.pdf
pdf_file_path2 = "./input/붙임1. 2024학년도 1학기 신편입생 수강신청 안내.pdf"

# 3. 붙임2. 2024학년도 2학기 수강신청 안내.pdf
pdf_file_path3 = (
    "./input/붙임1. 2024학년도 2학기 수강신청 안내.pdf"
)

# 4. 붙임3. 2023학년도 동계 계절학기 안내
pdf_file_path4 = "./input/붙임2. 2023학년도 동계 계절학기 실시 안내 공고.pdf"

# 5. 2024학년도 하계 계절학기 수강신청 안내
pdf_file_path5 = "./input/붙임2.2024학년도 하계 계절학기 실시 안내 공고_일정 수정_내용수정.pdf"

# 6. 2024학년도 1학기 수강지도 상담 안내
pdf_file_path6 = "./input/붙임3.2024-1학기 수강지도상담 실시 공고문.pdf"

# 7. 2024학년도 2학기 수강지도 상담 안내
pdf_file_path7 = "./input/붙임3.2024-2학기 수강지도상담 실시 공고문.pdf"

loader = PyPDFLoader(pdf_file_path7)
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=40)
docs = text_splitter.split_documents(pages)

lc_docs = []
for doc in docs:
    lc_docs.append(
        Document(
            page_content=doc.page_content.replace("\n", ""),
            metadata={"source": pdf_file_path7},
        )
    )

In [14]:
# 붙임1. 2024학년도 1학기 수강신청 안내
# allowed_nodes = [
#     "Course",               # 강의 관련 정보 (예: 강의명, 코드)
#     "Student",              # 학생 정보 (예: 학번, 이름)
#     "Department",           # 학과 정보 (예: 학과명)
#     "Semester",             # 학기 정보 (예: 2024년 1학기)
#     "RegistrationPeriod",   # 수강신청 기간
#     "Schedule",             # 강의 시간표 (예: 강의 요일, 시간)
#     "Professor",            # 교수 정보 (예: 교수명)
#     "Prerequisite",         # 선수 과목 정보
#     "Credit",               # 학점 정보
#     "Notice"                # 학사 공지
# ]

# allowed_relationships = [
#     "ENROLLED_IN",          # 학생 -> 강의 (학생이 특정 강의를 신청)
#     "OFFERED_BY",           # 강의 -> 학과 (특정 학과에서 제공하는 강의)
#     "TAUGHT_BY",            # 강의 -> 교수 (강의를 담당하는 교수)
#     "HAS_PREREQUISITE",     # 강의 -> 선수 과목 (강의가 요구하는 선수 과목)
#     "PART_OF",              # 강의 -> 학기 (특정 학기에 제공되는 강의)
#     "DURING",               # 수강신청 기간 -> 학기 (수강신청 기간과 학기 관계)
#     "BELONGS_TO",           # 학생 -> 학과 (학생이 속한 학과)
#     "HAS_CREDIT",           # 강의 -> 학점 (강의가 제공하는 학점)
#     "POSTED_NOTICE",        # 학과/학교 -> 공지 (학과 또는 학교에서 공지한 내용)
#     "SCHEDULED_ON"          # 강의 -> 시간표 (강의 시간표 정보)
# ]

# 붙임1. 2024학년도 1학기 신편입생 수강신청 안내
allowed_nodes2 = [
    "Student",  # 학생 정보 (예: 신입생, 편입생, 학번)
    "Course",  # 강의 관련 정보 (예: 강의명, 코드)
    "Semester",  # 학기 정보 (예: 2024년 1학기)
    "RegistrationPeriod",  # 수강신청 기간
    "Counseling",  # 수강지도상담 정보
    "Timetable",  # 강의 시간표 (예: 요일, 시간)
    "GraduationRequirement",  # 졸업요건 정보
    "Notice",  # 학사 공지
]

allowed_relationships2 = [
    "ENROLLED_IN",  # 학생 -> 강의 (학생이 특정 강의를 신청)
    "HAS_TIMETABLE",  # 강의 -> 시간표 (강의 시간표 정보)
    "PART_OF_SEMESTER",  # 강의 -> 학기 (특정 학기에 제공되는 강의)
    "WITHIN_REGISTRATION",  # 수강신청 기간 -> 학기 (수강신청 기간과 학기 관계)
    "REQUIRES_COUNSELING",  # 학생 -> 수강지도상담 (수강신청에 필요한 상담)
    "HAS_REQUIREMENT",  # 졸업요건 -> 학기/강의 (졸업 요건 관련 강의)
    "POSTED_NOTICE",  # 학교 -> 공지 (학교에서 공지한 내용)
]

# 붙임2. 2024학년도 2학기 수강신청 안내
allowed_nodes3 = [
    "Student",  # 학생 정보 (예: 재학생)
    "Department",  # 학과 정보 (예: 학과명)
    "Semester",  # 학기 정보 (예: 2024년 2학기)
    "RegistrationPeriod",  # 수강신청 기간
    "CourseBasket",  # 장바구니 시스템 (예: 장바구니 담기, 확인)
    "Credit",  # 학점 정보
    "Timetable",  # 강의 시간표 (예: 강의 요일, 시간)
    "Notice",  # 학사 공지
]

allowed_relationships3 = [
    "ENROLLED_IN",  # 학생 -> 강의 (학생이 특정 강의를 신청)
    "HAS_COURSE_BASKET",  # 학생 -> 장바구니 (학생의 장바구니 시스템 이용)
    "PART_OF_SEMESTER",  # 강의 -> 학기 (특정 학기에 제공되는 강의)
    "HAS_TIMETABLE",  # 강의 -> 시간표 (강의 시간표 정보)
    "WITHIN_REGISTRATION",  # 수강신청 기간 -> 학기 (수강신청 기간과 학기 관계)
    "POSTED_NOTICE",  # 학교/학과 -> 공지 (학교 또는 학과에서 공지한 내용)
    "CREDIT_LIMIT",  # 학기 -> 학점 (수강 신청 가능한 최대 학점)
]

# 붙임3. 2023학년도 동계 계절학기 안내
allowed_nodes4 = [
    "Student",  # 학생 정보 (예: 재학생, 휴학생)
    "Course",  # 강의 관련 정보 (예: 강의명, 코드)
    "Semester",  # 학기 정보 (예: 2023년 동계)
    "RegistrationPeriod",  # 수강신청 기간
    "RefundPolicy",  # 환불 규정 정보
    "Fee",  # 수강료 정보
    "Timetable",  # 강의 시간표 (예: 강의 요일, 시간)
    "Notice",  # 학사 공지
]

allowed_relationships4 = [
    "ENROLLED_IN",  # 학생 -> 강의 (학생이 특정 강의를 신청)
    "PART_OF_SEMESTER",  # 강의 -> 학기 (특정 학기에 제공되는 강의)
    "WITHIN_REGISTRATION",  # 수강신청 기간 -> 학기 (수강신청 기간과 학기 관계)
    "FEE_FOR_COURSE",  # 강의 -> 수강료 (특정 강의에 대한 수강료 정보)
    "FEE_REFUND_POLICY",  # 수강료 -> 환불 규정 (수강 취소 시 환불 규정)
    "HAS_TIMETABLE",  # 강의 -> 시간표 (강의 시간표 정보)
    "POSTED_NOTICE",  # 학교 -> 공지 (학교에서 공지한 내용)
]

# 2024학년도 하계 계절학기 수강신청 안내
allowed_nodes5 = [
    "Student",  # 학생 정보 (예: 재학생, 휴학생)
    "Course",  # 강의 관련 정보 (예: 강의명, 코드)
    "Semester",  # 학기 정보 (예: 2024년 하계)
    "RegistrationPeriod",  # 수강신청 기간
    "Fee",  # 수강료 정보
    "RefundPolicy",  # 환불 규정 정보
    "Timetable",  # 강의 시간표 (예: 강의 요일, 시간)
    "CancellationPeriod",  # 수강 취소 기간
    "Department",  # 학부/학과 정보
    "Notice",  # 학사 공지
    "LabCourse",  # 실습 과목 (예: 추진기관실습, 항공기 정비 실습)
    "InternshipCourse",  # 현장실습 과목 (예: 단기현장실습)
    "PaymentAccount",  # 납부 계좌 정보
]

allowed_relationships5 = [
    "ENROLLED_IN",  # 학생 -> 강의 (학생이 특정 강의를 신청)
    "PART_OF_SEMESTER",  # 강의 -> 학기 (특정 학기에 제공되는 강의)
    "WITHIN_REGISTRATION",  # 수강신청 기간 -> 학기 (수강신청 기간과 학기 관계)
    "REQUIRES_PAYMENT",  # 강의 -> 수강료 (특정 강의에 대한 수강료 정보)
    "FEE_REFUND_POLICY",  # 수강료 -> 환불 규정 (수강 취소 시 환불 규정)
    "HAS_TIMETABLE",  # 강의 -> 시간표 (강의 시간표 정보)
    "OFFERED_BY_DEPARTMENT",  # 강의 -> 학부/학과 (특정 학부/학과에서 제공되는 강의)
    "POSTED_NOTICE",  # 학교/학부 -> 공지 (학교 또는 학부에서 공지한 내용)
    "INCLUDES_LAB",  # 학기 -> 실습 과목 (실습 과목 포함)
    "OFFERS_INTERNSHIP",  # 학교 -> 현장실습 과목 (학교에서 제공하는 현장실습)
    "PAID_TO_ACCOUNT",  # 학생 -> 납부 계좌 (수강료 납부 계좌와 학생 관계)
    "CANCELLED_WITHIN",  # 수강 취소 -> 기간 (수강 취소 가능 기간)
    "ALLOWS_PARTIAL_REFUND",  # 환불 규정 -> 수강 취소 (일부 환불 규정)
]

# 2024학년도 1학기 수강지도 상담 안내
allowed_nodes6 = [
    "Student",  # 학생 정보 (예: 수강 예정 학부생, 휴학생 포함)
    "Course",  # 강의 관련 정보 (예: 주전공 과목)
    "Advisor",  # 지도교수 정보
    "Department",  # 학부/학과 정보
    "Semester",  # 학기 정보 (예: 2024년 1학기)
    "CounselingPeriod",  # 상담 기간
    "RegistrationPeriod",  # 수강신청 기간
    "CounselingSystem",  # 역량관리 시스템 정보
    "RequestStatus",  # 상담 신청 상태 (예: 승인, 반려)
    "Notice",  # 학사 공지
]

allowed_relationships6 = [
    "REQUIRES_COUNSELING",  # 학생 -> 수강지도 상담 (수강신청 전 상담 필수)
    "HAS_COURSE",  # 학생 -> 강의 (상담 대상 과목)
    "ASSIGNED_ADVISOR",  # 학생 -> 지도교수 (학생의 지도교수)
    "PART_OF_SEMESTER",  # 강의 -> 학기 (특정 학기에 개설된 강의)
    "DURING_COUNSELING",  # 상담 기간 -> 학기 (상담 기간과 학기 관계)
    "USING_SYSTEM",  # 학생/교수 -> 상담 시스템 (역량관리 시스템 사용)
    "APPROVED_BY_ADVISOR",  # 상담 신청 -> 지도교수 승인 (상담 내용 승인)
    "POSTED_NOTICE",  # 학교/학부 -> 공지 (학교 또는 학부에서 공지한 내용)
    "BELONGS_TO_DEPARTMENT",  # 학생 -> 학부 (학생 소속 학부)
    "HAS_STATUS",  # 상담 신청 -> 상태 (상담 신청 상태: 승인, 반려)
]

# 2024학년도 2학기 수강지도 상담 안내
allowed_nodes7 = [
    "Student",  # 학생 정보 (예: 수강 예정 학부생, 휴학생 포함)
    "Course",  # 강의 관련 정보 (예: 주전공 과목)
    "Advisor",  # 지도교수 정보
    "Department",  # 학부/학과 정보
    "Semester",  # 학기 정보 (예: 2024년 2학기)
    "CounselingPeriod",  # 상담 기간
    "RegistrationPeriod",  # 수강신청 기간
    "CounselingSystem",  # 역량관리 시스템 정보
    "RequestStatus",  # 상담 신청 상태 (예: 승인, 반려)
    "Notice",  # 학사 공지
]

allowed_relationships7 = [
    "REQUIRES_COUNSELING",  # 학생 -> 수강지도 상담 (수강신청 전 상담 필수)
    "HAS_COURSE",  # 학생 -> 강의 (상담 대상 과목)
    "ASSIGNED_ADVISOR",  # 학생 -> 지도교수 (학생의 지도교수)
    "PART_OF_SEMESTER",  # 강의 -> 학기 (특정 학기에 개설된 강의)
    "DURING_COUNSELING",  # 상담 기간 -> 학기 (상담 기간과 학기 관계)
    "USING_SYSTEM",  # 학생/교수 -> 상담 시스템 (역량관리 시스템 사용)
    "APPROVED_BY_ADVISOR",  # 상담 신청 -> 지도교수 승인 (상담 내용 승인)
    "POSTED_NOTICE",  # 학교/학부 -> 공지 (학교 또는 학부에서 공지한 내용)
    "BELONGS_TO_DEPARTMENT",  # 학생 -> 학부 (학생 소속 학부)
    "HAS_STATUS",  # 상담 신청 -> 상태 (상담 신청 상태: 승인, 반려)
]

In [31]:
# Transform documents into graph documents
transformer = LLMGraphTransformer(
    llm=llm_model,
    allowed_nodes=allowed_nodes7,
    allowed_relationships=allowed_relationships7,
    node_properties=False,
    relationship_properties=False,
)

In [32]:
graph_documents = transformer.convert_to_graph_documents(lc_docs)
graph.add_graph_documents(graph_documents, include_source=True)

In [37]:
# Use the stored connection parameters
index = Neo4jVector.from_existing_graph(
    embedding=embedding_model,
    url=neo4j_uri,
    username=neo4j_username,
    password=neo4j_password,
    database="neo4j",
    node_label="*",  # Adjust node_label as needed
    text_node_properties=["*"],
    embedding_node_property="embedding",
    index_name="vector_index",
    keyword_index_name="entity_index",
    search_type="hybrid",
)

In [38]:
# Retrieve the graph schema
schema = graph.get_schema

# Set up the QA chain
template = """
Task: Generate a Cypher statement to query the graph database.

Instructions:
Use only relationship types and properties provided in schema.
Do not use other relationship types or properties that are not provided.

schema:
{schema}

Note: Do not include explanations or apologies in your answers.
Do not answer questions that ask anything other than creating Cypher statements.
Do not include any text other than generated Cypher statements.

Question: {question}"""

question_prompt = PromptTemplate(
    template=template, input_variables=["schema", "question"]
)

qa = GraphCypherQAChain.from_llm(
    llm=llm_model,
    graph=graph,
    cypher_prompt=question_prompt,
    verbose=True,
    allow_dangerous_requests=True,
)

In [39]:
while True:
    question = input("질문을 입력해주세요 (종료하려면 '종료' 입력): ")

    # '종료' 입력 시 반복 종료
    if question.lower() == "종료":
        print("프로그램을 종료합니다.")
        break

    # 질문을 qa 객체에 전달하여 응답을 받습니다.
    response = qa({"query": question})

    # 응답의 필요한 부분을 출력합니다.
    print(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
I'm sorry, I can't assist with that request.


CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'I': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DATABASE', 'DEALLOCATE', 'DELETE', 'DENY', 'DETACH', 'DROP', 'DRYRUN', 'FINISH', 'GRANT', 'INSERT', 'LIMIT', 'MATCH', 'MERGE', 'NODETACH', 'OFFSET', 'OPTIONAL', 'REALLOCATE', 'REMOVE', 'RENAME', 'RETURN', 'REVOKE', 'ENABLE SERVER', 'SET', 'SHOW', 'SKIP', 'TERMINATE', 'UNWIND', 'USE' or 'WITH' (line 1, column 1 (offset: 0))
"I'm sorry, I can't assist with that request."
 ^}